# Introduction 

Use this dataset of airline arrival information to predict how late flights
will be. A flight only counts as late if it is more than 30 minutes late.
1. The project should follow guideline as previous projects.
2. Apply models in Naïve Bayes, Logistic Regression, Decision
Tree, Random Forest, Gradient Boosting and SVM.
3. Apply PCA, SelectKBest and RFE for feature selections.
4. Using gridsearchCV to obtain best parameters for models.
5. Compare performances among models, write up analysis
why the model is good or bad in the algorithmic approach
(explain why the algorithm is good or bad for the dataset
structure, can you do something to improve the model?)
6. Include the conclusions.

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
import time 
import datetime
import seaborn as sns

# A quick look of datasets

In [4]:
# load the data
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
data = pd.read_csv("./2008.csv")
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 1.5+ GB


Outlier Detection and Removal

In [7]:
pd.options.display.float_format = "{:.2f}".format

In [8]:
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.00,7009728.00,7009728.00,7009728.00,6873482.00,7009728.00,6858079.00,7009728.00,7009728.00,6855029.00,7008884.00,6855029.00,6855029.00,6873482.00,7009728.00,6858079.00,6872670.00,7009728.00,7009728.00,1524735.00,1524735.00,1524735.00,1524735.00,1524735.00
mean,2008.00,6.38,15.73,3.92,1333.83,1326.09,1481.26,1494.80,2224.20,127.32,128.87,104.02,8.17,9.97,726.39,6.86,16.45,0.02,0.00,15.77,3.04,17.16,0.07,20.77
std,0.00,3.41,8.80,1.99,478.07,464.25,505.23,482.67,1961.72,70.19,69.41,67.44,38.50,35.31,562.10,4.93,11.33,0.14,0.05,40.10,19.50,31.89,1.84,39.26
min,2008.00,1.00,1.00,1.00,1.00,0.00,1.00,0.00,1.00,12.00,-141.00,0.00,-519.00,-534.00,11.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2008.00,3.00,8.00,2.00,928.00,925.00,1107.00,1115.00,622.00,77.00,80.00,55.00,-10.00,-4.00,325.00,4.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,2008.00,6.00,16.00,4.00,1325.00,1320.00,1512.00,1517.00,1571.00,110.00,110.00,86.00,-2.00,-1.00,581.00,6.00,14.00,0.00,0.00,0.00,0.00,6.00,0.00,0.00
75%,2008.00,9.00,23.00,6.00,1728.00,1715.00,1909.00,1907.00,3518.00,157.00,159.00,132.00,12.00,8.00,954.00,8.00,19.00,0.00,0.00,16.00,0.00,21.00,0.00,26.00
max,2008.00,12.00,31.00,7.00,2400.00,2359.00,2400.00,2400.00,9743.00,1379.00,1435.00,1350.00,2461.00,2467.00,4962.00,308.00,429.00,1.00,1.00,2436.00,1352.00,1357.00,392.00,1316.00


Year is always 2008, we can drop Year column. 

In [12]:
data.drop(columns = 'Year', inplace=True)
data.head()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,3,4,2003.00,1955,2211.00,2225,WN,335,N712SW,128.00,150.00,116.00,-14.00,8.00,IAD,TPA,810,4.00,8.00,0,NaN,0,nan,nan,nan,nan,nan
1,1,3,4,754.00,735,1002.00,1000,WN,3231,N772SW,128.00,145.00,113.00,2.00,19.00,IAD,TPA,810,5.00,10.00,0,NaN,0,nan,nan,nan,nan,nan
2,1,3,4,628.00,620,804.00,750,WN,448,N428WN,96.00,90.00,76.00,14.00,8.00,IND,BWI,515,3.00,17.00,0,NaN,0,nan,nan,nan,nan,nan
3,1,3,4,926.00,930,1054.00,1100,WN,1746,N612SW,88.00,90.00,78.00,-6.00,-4.00,IND,BWI,515,3.00,7.00,0,NaN,0,nan,nan,nan,nan,nan
4,1,3,4,1829.00,1755,1959.00,1925,WN,3920,N464WN,90.00,90.00,77.00,34.00,34.00,IND,BWI,515,3.00,10.00,0,NaN,0,2.00,0.00,0.00,0.00,32.00


In [16]:
# Dữ liệu dạng object 
data.describe(include='O')

,UniqueCarrier,TailNum,Origin,Dest,CancellationCode
count,7009728,6926363,7009728,7009728,137434
unique,20,5373,303,304,4
top,WN,N476HA,ATL,ATL,B
freq,1201754,4701,414513,414521,54904


Now the remained Numeric features are used IQR to check the outliers.

In [14]:
# dataNumeric = data.select_dtypes(include=np.number)
# # Drop FlightNum, it dont have outlier
# dataNumeric.drop(columns = 'FlightNum', inplace=True)

# # get quartile 1st and 3rd
# Q1, Q3 = dataNumeric.quantile(0.25), dataNumeric.quantile(0.75) 
# IQR = Q3 - Q1
# # Boundary
# cutOff = 3*IQR
# # Setup Boudnary
# lower, upper = Q1 - cutOff, Q3 + cutOff # identify Outliers
# for i in dataNumeric:
#     print(dataNumeric[i][(dataNumeric[i] < lower[i]) | (dataNumeric[i] > upper[i])].value_counts().sort_index())
#     print('Total Outliers for ' + i + ' is ' , dataNumeric[i][(dataNumeric[i] < lower[i]) | (dataNumeric[i] > upper[i])].value_counts().sum())
#     print('--------------------------------------------------------------\n')

Check and Fill Missing Value

In [15]:
# Number of missing values in each column
missingValueColumns = (data.isnull().sum())
# Find missing column in data
missingValueColumnsFrame = missingValueColumns[missingValueColumns > 0].to_frame()
# Rename to 0 to Count missingValueColumnsFrame=missingValueColumnsFrame.rename(columns={0:'Count'})
# add percentage column
missingValueColumnsFrame['Percentage'] = missingValueColumnsFrame/data.shape[0] * 100
missingValueColumnsFrame

,0,Percentage
DepTime,136246,1.94
ArrTime,151649,2.16
TailNum,83365,1.19
ActualElapsedTime,154699,2.21
CRSElapsedTime,844,0.01
AirTime,154699,2.21
ArrDelay,154699,2.21
DepDelay,136246,1.94
TaxiIn,151649,2.16
TaxiOut,137058,1.96


In [ ]:
# Reference
